In [1]:
import dask.dataframe as ddf
import pandas as pd
import numpy as np
import re
import os
import time
from glob import glob
import pickle

/home/ubuntu/.local/lib/python3.8/site-packages/dask/dataframe/utils.py:374: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/home/ubuntu/.local/lib/python3.8/site-packages/dask/dataframe/utils.py:374: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/home/ubuntu/.local/lib/python3.8/site-packages/dask/dataframe/utils.py:374: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


In [ ]:
# Use 0 as the threshold if we want the results for all group pairs
threshold = 0
# threshold = 30

In [2]:
sr_path = '/home/ubuntu/scratch/xinrui/intermediate-result-new-select-editors/warring-camps/sr/'

In [3]:
sr = pd.read_parquet(sr_path)

In [4]:
with open('../../all-data-article-clustering/articles-100-threshold.pkl' ,'rb') as f:
    selected_articles = pickle.load(f) 

In [5]:
sr = sr[sr.index.get_level_values(1).isin(selected_articles)]

In [6]:
hl_path = '/home/ubuntu/scratch/xinrui/intermediate-result-new-select-editors/warring-camps/hl/'

In [7]:
hl = pd.read_parquet(hl_path)

In [8]:
hl = hl[hl['page.title'].isin(selected_articles)]

In [9]:
group_pairs = pd.read_csv('result/mutual-and-min-revert.csv', sep="\t",encoding='utf-16')

In [10]:
group_pairs = group_pairs.sort_values('min_revert',ascending=False)

In [2]:
kept_group_pairs = group_pairs[group_pairs['min_revert']>=threshold]

In [14]:
kept_group_pairs = kept_group_pairs.drop(columns=['mutual_revert','min_revert'])

In [15]:
df = pd.read_csv('result/group-pair-article-mutual-revert-info.csv', sep="\t",encoding='utf-16')

In [16]:
df = kept_group_pairs.merge(df,on=['group1','group2'])

In [17]:
user_group = pd.read_csv('result/all-data-editor-cluster-result.csv', sep="\t",encoding='utf-16').set_index('contributor.username')

In [18]:
sr = sr.reset_index()

In [19]:
sr = sr.merge(user_group,left_on='contributor.username',right_index=True)

In [20]:
group_sr = sr.groupby(['group','page.title']).agg({'all_added_tokens':'sum','survived_tokens':'sum'})

In [21]:
group_sr['sr'] = group_sr['survived_tokens'] / group_sr['all_added_tokens']

In [22]:
group_sr = group_sr[['sr']]

In [23]:
hl = hl.merge(user_group,left_on='contributor.username',right_index=True)

In [24]:
group_hl = hl.groupby(['group','page.title']).agg({'hl':'mean'})

In [25]:
df = df.merge(group_sr.reset_index(),left_on=['group1','page.title'],right_on=['group','page.title'],how='left').rename(columns={'sr':'sr_group1'})
df = df.merge(group_sr.reset_index(),left_on=['group2','page.title'],right_on=['group','page.title'],how='left').rename(columns={'sr':'sr_group2'})

In [26]:
df = df.drop(columns={'group_x','group_y'})

In [27]:
df = df.merge(group_hl.reset_index(),left_on=['group1','page.title'],right_on=['group','page.title'],how='left').rename(columns={'hl':'hl_group1'})
df = df.merge(group_hl.reset_index(),left_on=['group2','page.title'],right_on=['group','page.title'],how='left').rename(columns={'hl':'hl_group2'})

In [28]:
df = df.drop(columns={'group_x','group_y'})

In [29]:
df['sr_quotient'] = df[['sr_group1','sr_group2']][(~np.isnan(df['sr_group1'])) & (~np.isnan(df['sr_group2']))].\
min(axis=1) / df[['sr_group1','sr_group2']].max(axis=1) 

In [30]:
df['hl_quotient'] = df[['hl_group1','hl_group2']][(~np.isnan(df['hl_group1'])) & (~np.isnan(df['hl_group2']))].\
min(axis=1) / df[['hl_group1','hl_group2']].max(axis=1) 

In [31]:
df = df.drop(columns={'mutual_revert'})

In [ ]:
# store result with different names based on different thresholds. 
if treshold == 0:
    df.to_csv('result/sr-hl-for-all-group-pairs.csv', index=False,sep="\t",encoding='utf-16')
else:
    df.to_csv('result/sr-hl-for-top-ranked-pairs.csv', index=False,sep="\t",encoding='utf-16') 